In [1]:
import pandas as pd
import os

# Function to convert text files to CSV with space as delimiter
def txt_to_csv(file_path):
    csv_file_path = file_path.replace('.txt', '.csv')
    with open(file_path, 'r') as txt_file:
        lines = txt_file.readlines()
        with open(csv_file_path, 'w') as csv_file:
            for line in lines:
                csv_file.write(','.join(line.split()) + '\n')

# List of folder paths for each city along with city names
city_folders = [('chi_daily_average_febmarchapril', 'Chicago'), 
                ('mia_daily_average_febmarchapril', 'Miami'), 
                ('nyc_daily_average_febmarchapril', 'New York'), 
                ('aus_daily_average_febmarchapril', 'Austin')]

# Initialize an empty list to store all DataFrames
all_dataframes = []

# Determine the number of days in March 2024
current_year = 2024
current_month = 3
num_days_march_2024 = 18

# Iterate over each city folder
for folder_path, city_name in city_folders:
    # Iterate over each file in the folder
    for file_name in os.listdir(folder_path):
        # Convert text file to CSV
        if file_name.endswith('.txt'):
            txt_file_path = os.path.join(folder_path, file_name)
            txt_to_csv(txt_file_path)
            file_name = file_name.replace('.txt', '.csv')
        
        # Extract month information from the file name
        file_name_parts = file_name.split('_')[1].split('-')
        year, month = file_name_parts[0], file_name_parts[1].replace('.csv', '')
        month_abbr = pd.to_datetime(month, format='%m').strftime('%b')
        
        # Determine the number of days in the current month
        if month == '02':
            num_days = 28 if not (int(year) % 4 == 0 and (int(year) % 100 != 0 or int(year) % 400 == 0)) else 29  # Leap year check
        elif month in ['04', '06', '09', '11']:
            num_days = 30
        elif year == str(current_year) and month == str(current_month):
            num_days = num_days_march_2024
        else:
            num_days = 31

        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(folder_path, file_name))
        
        df = df.iloc[:, :3]
        df = df.iloc[num_days + 3:, :]
        
        # Skip lines containing "Min", "Max", "Avg"
        df = df[~df.iloc[:, 0].str.contains('Min|Max|Avg|Total')]
        
        # Temperature DataFrame
        temp_df = df.iloc[0:num_days, 0:3].reset_index(drop=True)
        temp_df.columns = ['Max_Temp', 'Avg_Temp', 'Min_Temp']

        # Dew Point DataFrame
        dew_df = df.iloc[num_days:2*num_days, 0:3].reset_index(drop=True)
        dew_df.columns = ['Max_Dew', 'Avg_Dew', 'Min_Dew']

        # Humidity DataFrame
        humidity_df = df.iloc[2*num_days:3*num_days, 0:3].reset_index(drop=True)
        humidity_df.columns = ['Max_Humidity', 'Avg_Humidity', 'Min_Humidity']

        # Wind Speed DataFrame
        wind_df = df.iloc[3*num_days:4*num_days, 0:3].reset_index(drop=True)
        wind_df.columns = ['Max_Wind', 'Avg_Wind', 'Min_Wind']

        # Pressure DataFrame
        pressure_df = df.iloc[4*num_days:5*num_days, 0:3].reset_index(drop=True)
        pressure_df.columns = ['Max_Pressure', 'Avg_Pressure', 'Min_Pressure']
        
        # Combine the data for each feature into one DataFrame
        file_combined_df = pd.concat([temp_df, dew_df, humidity_df, wind_df, pressure_df], axis=1)
        
        # Add a date column
        file_combined_df['Date'] = pd.date_range(start=f"{year}-{month_abbr}-01", periods=len(file_combined_df), freq='D')
        
        # Add city column (one-hot encoded)
        file_combined_df[city_name] = 1
        
        # Append the combined DataFrame to the list
        all_dataframes.append(file_combined_df)

# Concatenate all DataFrames in the list to create one big DataFrame
big_dataframe = pd.concat(all_dataframes, ignore_index=True)

# Fill NaN values in city columns with 0 (indicating the city is not the current one)
big_dataframe = big_dataframe.fillna(0)

# Save the combined DataFrame to a CSV file
big_dataframe.to_csv('combined_data_with_city.csv', index=False)


In [3]:
big_dataframe.columns

Index(['Max_Temp', 'Avg_Temp', 'Min_Temp', 'Max_Dew', 'Avg_Dew', 'Min_Dew',
       'Max_Humidity', 'Avg_Humidity', 'Min_Humidity', 'Max_Wind', 'Avg_Wind',
       'Min_Wind', 'Max_Pressure', 'Avg_Pressure', 'Min_Pressure', 'Date',
       'Chicago', 'Miami', 'New York', 'Austin'],
      dtype='object')

,Max_Temp,Avg_Temp,Min_Temp,Max_Humidity,Avg_Humidity,Min_Humidity,Max_Wind,Avg_Wind,Min_Wind,Chicago,Miami,New York,Austin,Max_Temp_Next_Day,Date
0,54,43.3,35,82,64.8,45,23,15.0,3,1.0,0.0,0.0,0.0,67,2020-02-01
1,67,61.3,55,75,58.8,42,14,7.6,0,0.0,1.0,0.0,0.0,77,2020-02-01
2,77,56.3,34,89,51.9,20,18,7.1,0,0.0,0.0,0.0,1.0,43,2020-02-01
3,43,38.1,35,87,75.9,47,15,8.7,3,0.0,0.0,1.0,0.0,54,2020-02-01
4,54,43.3,35,82,64.8,45,23,15.0,3,1.0,0.0,0.0,0.0,56,2020-02-01
